In [70]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

ventas_2018 = pd.read_csv('data/ventas_2018.csv', encoding = "ISO-8859-1")
ventas_2019 = pd.read_csv('data/ventas_2019.csv', encoding = "ISO-8859-1")
ventas_2020 = pd.read_csv('data/ventas_2020.csv', encoding = "ISO-8859-1")

ventas_2018['PERIODO'] = ventas_2018.apply(lambda x: date(2018, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2018 = ventas_2018.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2019['PERIODO'] = ventas_2019.apply(lambda x: date(2019, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2019 = ventas_2019.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2020['PERIODO'] = ventas_2020.apply(lambda x: date(2020, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2020 = ventas_2020.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2018 = ventas_2018.astype({'PERIODO': 'datetime64[ns]'})

ventas_2019 = ventas_2019.astype({'PERIODO': 'datetime64[ns]'})

ventas_2020 = ventas_2020.astype({'PERIODO': 'datetime64[ns]'})

with open('data/skus_filter.json', 'r') as file:
    skus_filter = json.load(file)

ventas_finales = pd.concat([ventas_2018, ventas_2019, ventas_2020]).sort_values('PERIODO')

ventas_finales = ventas_finales[~ventas_finales['ID_SKU_VENTA'].isin(skus_filter)]
ventas_finales = ventas_finales.drop(columns=['DESCR_PROD'], axis=1)

confianza = 1.65

hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'CERVECERA': 'SANTIAGO SUR',
 'MODELO': 'SANTIAGO SUR',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}

ventas_finales = ventas_finales[ventas_finales['DESCR_CENDIST'].isin(list(hubs_dict.keys()))]
ventas_finales = ventas_finales.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

In [4]:
ventas_finales.groupby(by=['DESCR_CENDIST', 'PERIODO']).sum()['SUM(PREVENTA)'].reset_index()

,DESCR_CENDIST,PERIODO,SUM(PREVENTA)
0,ANTOFAGASTA,2018-01-01,473498.941670
1,ANTOFAGASTA,2018-02-01,460073.616663
2,ANTOFAGASTA,2018-03-01,498401.766670
3,ANTOFAGASTA,2018-04-01,417496.166666
4,ANTOFAGASTA,2018-05-01,421189.233333
...,...,...,...
763,VALDIVIA,2020-04-01,182394.000000
764,VALDIVIA,2020-05-01,165507.000000
765,VALDIVIA,2020-06-01,146369.000000
766,VALDIVIA,2020-07-01,173674.000000


In [85]:
len(ventas_finales[(ventas_finales['ID_SKU_VENTA'] == 870195) & (ventas_finales['DESCR_CENDIST'] == 'SANTIAGO SUR')])

32

## Merge para pasar de cajas a pallets

In [ ]:
## code

# Sin agrupar

In [71]:
means_stddev_dict = {}
ids = ventas_finales['ID_SKU_VENTA'].drop_duplicates()
cds = ventas_finales['DESCR_CENDIST'].drop_duplicates()

for id_ in ids:
    
    data_id = ventas_finales[ventas_finales['ID_SKU_VENTA'] == id_]
    
    for cd in cds:
        
        if pd.isnull(cd):
            continue
        
        data_id_cd = data_id[data_id['DESCR_CENDIST'] == cd]
        
        #data_id_cd_semana = data_id_cd
        #data_id_cd_semana['SUM(PREVENTA)'] = data_id_cd_semana.apply(lambda x: math.ceil(x['SUM(PREVENTA)'] / 4), axis=1)
        ventas = data_id_cd['SUM(PREVENTA)'].to_numpy()
        
        largo = len(ventas)
        
        if largo == 32:
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        else:
            excedente = 32 - largo
            for i in range(excedente):
                ventas = np.append(ventas, [0])
            mean = ventas.mean() / 4
            std = ventas.std() / 4  
        
        if pd.isnull(mean):
            means_stddev_dict[(id_, cd)] = (0, 0)
            continue

        means_stddev_dict[(id_, cd)] = (mean, std)

In [59]:
cds

0        ANTOFAGASTA
32             ARICA
64            CALAMA
96            CASTRO
128        CERVECERA
160          CHILLAN
192          COPIAPO
224         COQUIMBO
256        COYHAIQUE
288          ILLAPEL
320          IQUIQUE
352        LLAY LLAY
383      LOS ANGELES
415           OSORNO
447           OVALLE
479     PUERTO MONTT
511     SANTIAGO SUR
543            TALCA
575       TALCAHUANO
607           TEMUCO
639         VALDIVIA
959          CURAUMA
1119          MODELO
1236        RANCAGUA
Name: DESCR_CENDIST, dtype: object

In [3]:
means_stddev_dict

{(515, 'ANTOFAGASTA'): (44.634765625, 7.770042590560934),
 (515, 'ARICA'): (4.2265625, 1.7347092864782128),
 (515, 'CALAMA'): (23.3203125, 8.811000398782408),
 (515, 'CASTRO'): (13.875, 8.575947469521953),
 (515, 'CERVECERA'): (650.3671875, 247.04143902433626),
 (515, 'CHILLAN'): (22.9375, 10.341814395936527),
 (515, 'COPIAPO'): (24.421875, 6.64984653840786),
 (515, 'COQUIMBO'): (77.2578125, 25.375806974160323),
 (515, 'COYHAIQUE'): (6.90625, 2.527775393008643),
 (515, 'ILLAPEL'): (10.15625, 5.825171000708906),
 (515, 'IQUIQUE'): (7.8515625, 4.301098776312136),
 (515, 'LLAY LLAY'): (1.6953125, 1.0784363228043417),
 (515, 'LOS ANGELES'): (58.578125, 22.618562404237256),
 (515, 'OSORNO'): (25.3515625, 9.428360511435365),
 (515, 'OVALLE'): (13.3125, 7.3529862130973695),
 (515, 'PUERTO MONTT'): (43.1484375, 11.140063428167442),
 (515, 'SANTIAGO SUR'): (219.390625, 74.59400365216614),
 (515, 'TALCA'): (115.4921875, 38.49033382993507),
 (515, 'TALCAHUANO'): (166.1171875, 51.835564241308724),

In [4]:
with open('data/mean_std_coefvar_sku_cd.csv', 'w') as file:
    
    file.write('CD' + ',' + 'SKU' + ',' + 'MEAN' + ',' + 'STD' + ',' + 'COEFVAR' + '\n')
    
    for key in means_stddev_dict:
        
        file.write(str(key[1]) + ',' + str(key[0]) + ',' + str(means_stddev_dict[key][0]) + ',' + str(means_stddev_dict[key][1]) + ',' + str(means_stddev_dict[key][1] / means_stddev_dict[key][0]) + '\n')

<ipython-input-4-35a25e0462ca>:7: RuntimeWarning: invalid value encountered in double_scalars
  file.write(str(key[1]) + ',' + str(key[0]) + ',' + str(means_stddev_dict[key][0]) + ',' + str(means_stddev_dict[key][1]) + ',' + str(means_stddev_dict[key][1] / means_stddev_dict[key][0]) + '\n')


In [72]:
s_stock_dict = {}

for key in means_stddev_dict:
    
    s_stock_dict[key] = means_stddev_dict[key][0] + (confianza * means_stddev_dict[key][1])

# Agrupado

In [73]:
ventas_finales_new = ventas_finales.copy()

In [74]:
sheets_dict = pd.read_excel('data/sku_unification_noR.xlsx', sheet_name=None)

master = pd.DataFrame()
for name in sheets_dict:
    
    df = sheets_dict[name]
    master = pd.concat([master, df])
    
dict_sku_cd = {}
master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})

for index, row in master.iterrows():
    
    if row['CANTIDAD HUB 3R'] != 0:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1
        
    else:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0

In [75]:
dict_sku_cd

{(7585, 'ARICA'): 1,
 (871008, 'ARICA'): 0,
 (450684, 'ARICA'): 1,
 (450688, 'ARICA'): 0,
 (442125, 'ARICA'): 1,
 (450776, 'ARICA'): 1,
 (450830, 'ARICA'): 1,
 (450837, 'ARICA'): 1,
 (700279, 'ARICA'): 0,
 (875168, 'ARICA'): 0,
 (700281, 'ARICA'): 0,
 (700282, 'ARICA'): 0,
 (700337, 'ARICA'): 1,
 (700406, 'ARICA'): 1,
 (700438, 'ARICA'): 1,
 (442019, 'ARICA'): 1,
 (1697, 'ARICA'): 0,
 (443131, 'ARICA'): 1,
 (871006, 'ARICA'): 0,
 (700594, 'ARICA'): 0,
 (445309, 'ARICA'): 0,
 (870976, 'ARICA'): 1,
 (445757, 'ARICA'): 0,
 (445746, 'ARICA'): 0,
 (445807, 'ARICA'): 0,
 (445820, 'ARICA'): 0,
 (445823, 'ARICA'): 0,
 (445729, 'ARICA'): 0,
 (445844, 'ARICA'): 1,
 (446111, 'ARICA'): 1,
 (1668, 'ARICA'): 1,
 (446125, 'ARICA'): 1,
 (870992, 'ARICA'): 1,
 (870853, 'ARICA'): 1,
 (450305, 'ARICA'): 1,
 (450409, 'ARICA'): 1,
 (700492, 'ARICA'): 0,
 (700437, 'ARICA'): 1,
 (269124, 'ARICA'): 0,
 (870108, 'ARICA'): 0,
 (268758, 'ARICA'): 0,
 (445793, 'ARICA'): 1,
 (268528, 'ARICA'): 0,
 (445812, 'ARICA'

## MODIFICAR DATAFRAME SEGÚN DICT Y AGRUPAR POR SKU, CD Y FECHA

In [76]:
## code
ventas_finales_new['DESCR_CENDIST'] = ventas_finales_new.apply(lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 else hubs_dict[x['DESCR_CENDIST']], axis=1)
ventas_finales_new = ventas_finales_new.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

In [35]:
ventas_finales_new[(ventas_finales_new['ID_SKU_VENTA'] == 875365) & (ventas_finales_new['DESCR_CENDIST'] == 'CURAUMA')]

,ID_SKU_VENTA,DESCR_CENDIST,PERIODO,SUM(PREVENTA)
649306,875365,CURAUMA,2018-01-01,1.0
649307,875365,CURAUMA,2018-03-01,1.0


In [77]:
means_stddev_dict_optimized = {}
ids = ventas_finales_new['ID_SKU_VENTA'].drop_duplicates()
cds = ventas_finales_new['DESCR_CENDIST'].drop_duplicates()

counter = 0

for id_ in ids:
    
    data_id = ventas_finales_new[ventas_finales_new['ID_SKU_VENTA'] == id_]
    
    for cd in cds:
        
        if pd.isnull(cd):
            continue
        
        data_id_cd = data_id[data_id['DESCR_CENDIST'] == cd]
        
        if data_id_cd.empty:
            counter += 1
        
        #data_id_cd_semana = data_id_cd
        #data_id_cd_semana['SUM(PREVENTA)'] = data_id_cd_semana.apply(lambda x: math.ceil(x['SUM(PREVENTA)'] / 4), axis=1)
        
        ### MODIFICAR MEDIAS Y STDDEV: MODIFICAR DATAFRAME Y DPS AGRUPAR CHAO DIEGO FUMADOOOO
        ventas = data_id_cd['SUM(PREVENTA)'].to_numpy()
        
        largo = len(ventas)
        
        if largo == 32:
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        else:
            excedente = 32 - largo
            for i in range(excedente):
                ventas = np.append(ventas, [0])
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        if pd.isnull(mean):
            means_stddev_dict_optimized[(id_, cd)] = (0, 0)
            continue

        means_stddev_dict_optimized[(id_, cd)] = (mean, std)
        
print(counter)

4235


In [69]:
cds

0       ANTOFAGASTA
32          COPIAPO
64         COQUIMBO
96        COYHAIQUE
128         CURAUMA
160         IQUIQUE
192    PUERTO MONTT
224    SANTIAGO SUR
256           TALCA
288      TALCAHUANO
320          TEMUCO
352        VALDIVIA
Name: DESCR_CENDIST, dtype: object

In [78]:
len(means_stddev_dict)

40608

In [79]:
len(means_stddev_dict_optimized)

20304

In [28]:
means_stddev_dict_optimized

{(515, 'ANTOFAGASTA'): (44.634765625, 7.770042590560934),
 (515, 'ARICA'): (4.2265625, 1.7347092864782128),
 (515, 'CALAMA'): (23.3203125, 8.811000398782408),
 (515, 'CASTRO'): (13.875, 8.575947469521953),
 (515, 'CERVECERA'): (650.3671875, 247.04143902433626),
 (515, 'CHILLAN'): (22.9375, 10.341814395936527),
 (515, 'COPIAPO'): (24.421875, 6.64984653840786),
 (515, 'COQUIMBO'): (77.2578125, 25.375806974160323),
 (515, 'COYHAIQUE'): (6.90625, 2.527775393008643),
 (515, 'ILLAPEL'): (10.15625, 5.825171000708906),
 (515, 'IQUIQUE'): (7.8515625, 4.301098776312136),
 (515, 'LLAY LLAY'): (1.6953125, 1.0784363228043417),
 (515, 'LOS ANGELES'): (58.578125, 22.618562404237256),
 (515, 'OSORNO'): (25.3515625, 9.428360511435365),
 (515, 'OVALLE'): (13.3125, 7.3529862130973695),
 (515, 'PUERTO MONTT'): (43.1484375, 11.140063428167442),
 (515, 'SANTIAGO SUR'): (219.390625, 74.59400365216614),
 (515, 'TALCA'): (115.4921875, 38.49033382993507),
 (515, 'TALCAHUANO'): (166.1171875, 51.835564241308724),

In [80]:
s_stock_dict_optimized = {}

for key in means_stddev_dict_optimized:
    
    s_stock_dict_optimized[key] = means_stddev_dict_optimized[key][0] + (confianza * means_stddev_dict_optimized[key][1])

In [30]:
s_stock_dict_optimized

{(515, 'ANTOFAGASTA'): 57.45533589942554,
 (515, 'ARICA'): 7.088832822689051,
 (515, 'CALAMA'): 37.85846315799097,
 (515, 'CASTRO'): 28.02531332471122,
 (515, 'CERVECERA'): 1057.9855618901547,
 (515, 'CHILLAN'): 40.001493753295264,
 (515, 'COPIAPO'): 35.39412178837297,
 (515, 'COQUIMBO'): 119.12789400736453,
 (515, 'COYHAIQUE'): 11.07707939846426,
 (515, 'ILLAPEL'): 19.767782151169694,
 (515, 'IQUIQUE'): 14.948375480915024,
 (515, 'LLAY LLAY'): 3.474732432627164,
 (515, 'LOS ANGELES'): 95.89875296699148,
 (515, 'OSORNO'): 40.90835734386835,
 (515, 'OVALLE'): 25.444927251610657,
 (515, 'PUERTO MONTT'): 61.52954215647628,
 (515, 'SANTIAGO SUR'): 342.47073102607413,
 (515, 'TALCA'): 179.00123831939285,
 (515, 'TALCAHUANO'): 251.6458684981594,
 (515, 'TEMUCO'): 90.5257365372258,
 (515, 'VALDIVIA'): 28.555836610950735,
 (515, 'CURAUMA'): 0.0,
 (515, 'MODELO'): 0.0,
 (515, 'RANCAGUA'): 0.0,
 (592, 'ANTOFAGASTA'): 337.1813359297991,
 (592, 'ARICA'): 144.82512129423344,
 (592, 'CALAMA'): 92.15

## RECORRER DICT Y PONER COMO 0 LOS S DE LOS (sku, cd) QUE NO ESTÉN EN EL DICT

In [ ]:
## code


In [81]:
diff_dict = {}

for key in s_stock_dict:
    
    if key in s_stock_dict_optimized:
        
        diff_dict[key] = s_stock_dict_optimized[key] - s_stock_dict[key]
        
    else:
        
        diff_dict[key] = - s_stock_dict[key]
        

In [82]:
diff_dict

{(515, 'ANTOFAGASTA'): 30.89028645485616,
 (515, 'ARICA'): -7.088832822689051,
 (515, 'CALAMA'): -37.85846315799097,
 (515, 'CASTRO'): -28.02531332471122,
 (515, 'CERVECERA'): -1057.9855618901547,
 (515, 'CHILLAN'): -40.001493753295264,
 (515, 'COPIAPO'): 0.0,
 (515, 'COQUIMBO'): 18.92084083612029,
 (515, 'COYHAIQUE'): 0.0,
 (515, 'ILLAPEL'): -19.767782151169694,
 (515, 'IQUIQUE'): 5.634553674712896,
 (515, 'LLAY LLAY'): -3.474732432627164,
 (515, 'LOS ANGELES'): -95.89875296699148,
 (515, 'OSORNO'): -40.90835734386835,
 (515, 'OVALLE'): -25.444927251610657,
 (515, 'PUERTO MONTT'): 25.947453312382564,
 (515, 'SANTIAGO SUR'): 1037.767669715562,
 (515, 'TALCA'): 0.0,
 (515, 'TALCAHUANO'): 130.6873404256166,
 (515, 'TEMUCO'): 0.0,
 (515, 'VALDIVIA'): 39.96088179286495,
 (515, 'CURAUMA'): 22.417150001493184,
 (515, 'MODELO'): -0.0,
 (515, 'RANCAGUA'): -0.0,
 (592, 'ANTOFAGASTA'): 86.52366450197684,
 (592, 'ARICA'): -144.82512129423344,
 (592, 'CALAMA'): -92.15079657396477,
 (592, 'CASTRO')

In [83]:
inventory_save_dict_sku = {}
    
for key in diff_dict:
        
    
    if key[0] in inventory_save_dict_sku:
            inventory_save_dict_sku[key[0]] += diff_dict[key]
                
    else:
            inventory_save_dict_sku[key[0]] = diff_dict[key]

In [111]:
inventory_save_dict_sku

{515: -44.22804088149995,
 592: -414.0439122280545,
 595: -1003.9201587080933,
 622: -108.45343462005098,
 763: -30.803106374919636,
 765: -228.42887383515819,
 799: -50.518776764653836,
 810: -63.072348872828,
 816: -58.96317559319749,
 817: -458.50841710230617,
 826: -49.167413303056094,
 829: -545.6807887781905,
 833: -79.45170284517411,
 856: -41.33591573842915,
 859: -607.9251779893154,
 905: -305.6949841909037,
 909: -614.2251360590509,
 914: -150.7247883362606,
 915: -494.92626852701005,
 919: -637.4328320431218,
 924: -122.93418380622097,
 925: -463.4859864319874,
 929: -650.2428768436391,
 934: -31.732059071925676,
 935: -289.33174571604513,
 939: -572.4363658213704,
 953: -764.5538087878447,
 957: -531.3594448911721,
 1005: -485.93313037455914,
 1007: -766.414260300784,
 1014: -182.48662164087725,
 1016: -167.37612073885708,
 1018: -577.6820688689152,
 1034: -61.388310515752295,
 1036: -144.54670288348547,
 1038: -91.14857402821346,
 1040: -173.8904690791884,
 1057: -98.29261

In [114]:
conversion_unidades = pd.read_csv('data/conversiones_unidades.csv', encoding = "ISO-8859-1")

In [115]:
conversion_unidades

,ID_SKU_VENTA,DESCR_SKU_LONG,CAJAS_POR_PALLET,CAJAS_POR_CAPA,CAPAS_X_PALLET
0,509,ESCUDO VNR1000X12-TR,56,8,7
1,510,AUSTRAL 6PX4-LAT350,100,10,10
2,511,AUSTRAL 6PX4-VNR330-SO,60,12,5
3,512,AUSTRAL VNR1000X12-TR,56,14,4
4,515,AUSTRAL VRE1000X12-TR,40,10,4
...,...,...,...,...,...
9849,999992,BOMBILLAS .,9801,99,99
9850,999995,VASO VIDRIO .,9801,99,99
9851,999996,VASO PLAST.285CC GAS CAJA P.CERV,9801,99,99
9852,999998,VASO PLAS. 400 .,9801,99,99


In [119]:
data_dict = {'id_sku': list(inventory_save_dict_sku.keys()), 'cantidad': list(inventory_save_dict_sku.values())}
data = pd.DataFrame.from_dict(data_dict)
conversion = pd.read_csv('data/conversiones_unidades.csv', encoding = "ISO-8859-1")[['ID_SKU_VENTA', 'CAJAS_POR_PALLET']]
data = data.merge(precio, left_on='id_sku', right_on='ID_SKU_VENTA').drop(columns='ID_SKU_VENTA')

In [126]:
precio

,ID_SKU_VENTA,CAJAS_POR_PALLET
0,509,56
1,510,100
2,511,60
3,512,56
4,515,40
...,...,...
9849,999992,9801
9850,999995,9801
9851,999996,9801
9852,999998,9801


In [121]:
data['cantidad_pallets'] = data.apply(lambda x: x['cantidad'] / x['CAJAS_POR_PALLET'], axis=1)

In [124]:
sku_pallets = {}

for index, row in data.iterrows():
    
    sku_pallets[row['id_sku']] = row['cantidad_pallets']

In [125]:
sku_pallets

{515.0: -1.1057010220374988,
 592.0: -10.351097805701363,
 595.0: -10.039201587080932,
 622.0: -13.556679327506373,
 763.0: -0.6160621274983927,
 765.0: -5.710721845878955,
 799.0: -1.0103755352930768,
 810.0: -1.26144697745656,
 816.0: -1.1792635118639498,
 817.0: -11.462710427557655,
 826.0: -0.9833482660611219,
 829.0: -13.642019719454762,
 833.0: -1.9862925711293529,
 856.0: -0.826718314768583,
 859.0: -15.198129449732885,
 905.0: -3.8211873023862966,
 909.0: -6.979831091580124,
 914.0: -3.768119708406515,
 915.0: -6.186578356587626,
 919.0: -6.374328320431219,
 924.0: -3.073354595155524,
 925.0: -5.793574830399843,
 929.0: -6.502428768436391,
 934.0: -0.6346411814385136,
 935.0: -3.6166468214505643,
 939.0: -6.504958702515572,
 953.0: -7.645538087878447,
 957.0: -6.641993061139651,
 1005.0: -6.074164129681989,
 1007.0: -7.66414260300784,
 1014.0: -1.8248662164087726,
 1016.0: -2.7896020123142846,
 1018.0: -6.5645689644194904,
 1034.0: -1.5347077628938073,
 1036.0: -3.6136675720871

In [ ]:
data_dict = {'id_sku': list(inventory_save_dict_sku.keys()), 'cantidad': list(inventory_save_dict_sku.values())}
data = pd.DataFrame.from_dict(data_dict)
precio = pd.read_csv('data/valor_por_sku.csv', encoding = "ISO-8859-1")[['SKU', 'Precio Caja']]
data = data.merge(precio, left_on='id_sku', right_on='SKU')

In [110]:
data.apply(lambda x: x['cantidad'] * x['Precio Caja'], axis=1).sum()

-1825039349.8740585

In [117]:
with open('data/delta_s_por_sku.csv', 'w') as file:
    file.write('ID_SKU_VENTA' + ',' + 'SUM(PREVENTA)' + '\n')
    
    for key in inventory_save_dict:
        file.write(str(key) + ',' + str(inventory_save_dict[key]) + '\n')
    
    

In [22]:
with open('data/delta_s_por_sku_por_cd.csv', 'w') as file:
    file.write('ID_SKU_VENTA' + ',' + 'DESCR_CENDIST' + ',' + 'SUM(PREVENTA)' + '\n')
    
    for key in diff_dict:
        file.write(str(key[0]) + ',' + str(key[1]) + ',' + str(diff_dict[key]) + '\n')